In [31]:
import requests
import json
import time
import os
from pprint import pprint
import pandas as pd
pd.set_option('display.precision', 1)

from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/scrape/bin/python'

In [32]:
pwd

'/Users/edwardterrell/Desktop/Training/kindle/strava'

In [33]:
from token_manager import get_valid_access_token
import requests

access_token = get_valid_access_token()
print("Using access token:", access_token)

# Make an authenticated API call
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)
print(response.json())

Using access token: 9c35e4fa37dd567b30f209628e9c8f976c8f88f8
{'id': 723727, 'username': 'eterrell', 'resource_state': 2, 'firstname': 'Ed', 'lastname': 'Terrell', 'bio': '', 'city': 'Boulder', 'state': 'CO', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2012-07-05T00:04:09Z', 'updated_at': '2025-06-19T23:51:05Z', 'badge_type_id': 1, 'weight': 61.235, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/large.jpg', 'friend': None, 'follower': None}


In [ ]:
# USED ONLY IF token_manager.py isn't generating correctly
client_id = 167221
client_secret = 'f4df8e0dec7d9d8e725707eaf05037fcc845989d'
auth_code = 'd973f0f86d2e56c3e7ac369809ca6b14fc814a98' # stored from last time

response = requests.post(
    'https://www.strava.com/oauth/token',
    data={
        'client_id': client_id,
        'client_secret': client_secret,
        'code': auth_code,
        'grant_type': 'authorization_code',
    }
)
# Check and save response
if response.status_code == 200:
    tokens = response.json()
    print("✅ Tokens received:")
    print(json.dumps(tokens, indent=2))

    # Save to tokens.json for use in token_manager.py
    with open("tokens.json", "w") as f:
        json.dump(tokens, f)
else:
    print("❌ Failed to get tokens:", response.status_code)
    print(response.text)

In [5]:
import datetime
expires_at = 1751931129
# Convert to datetime object (local time)
dt_local = datetime.datetime.fromtimestamp(expires_at)
print("Local time: ", dt_local)

Local time:  2025-07-07 17:32:09


In [34]:
# Get recent activities
response = requests.get(
    'https://www.strava.com/api/v3/athlete/activities',
    headers={'Authorization': f'Bearer {access_token}'}
)
activities = response.json()
response.status_code

200

In [35]:
# Show the first 3 activities with ID and name
for activity in activities[:3]:
    print(f"ID: {activity['id']}, Name: {activity['name']}, Date: {activity['start_date']}")

ID: 15048399185, Name: Jamestown and eats at Lucky's Bakery, Date: 2025-07-08T14:44:02Z
ID: 15037283785, Name: Old Stage, Date: 2025-07-07T12:38:38Z
ID: 15027982989, Name: North dirt roads to Longmont bike paths and Babettes, Date: 2025-07-06T14:14:42Z


### Proof of concept for activity_id

In [36]:
# activity ID you want to inspect
activity_id = 15048399185

# Define endpoint URL
url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
# Set up Authorization header and make request
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    zones = response.json()
    #pprint(zones[0])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
zone_series

suffer_score = pd.DataFrame(zones[0]).iloc[0,0]
print(suffer_score)
time_in_zones = zone_series.apply(lambda z: z['time']/60)
time_in_zones

101.0


0     21.4
1    109.4
2     26.1
3      0.8
4      0.0
Name: distribution_buckets, dtype: float64

In [37]:
# Build out strava_df with only the necessary columns
strava_df = pd.DataFrame(activities)
cols_needed = ['id','name', 'distance', 'moving_time','total_elevation_gain', 'sport_type',
          'average_speed', 'average_heartrate','max_heartrate', 'suffer_score','start_date']
strava_df = strava_df.loc[:,cols_needed]
strava_df.head(3)

,id,name,distance,moving_time,total_elevation_gain,sport_type,average_speed,average_heartrate,max_heartrate,suffer_score,start_date
0,15048399185,Jamestown and eats at Lucky's Bakery,60380.4,9467,757.0,Ride,6.4,110.9,140.0,101.0,2025-07-08T14:44:02Z
1,15037283785,Old Stage,44640.8,7128,430.0,Ride,6.3,109.6,134.0,54.0,2025-07-07T12:38:38Z
2,15027982989,North dirt roads to Longmont bike paths and Ba...,59511.9,10021,361.6,Ride,5.9,102.5,129.0,47.0,2025-07-06T14:14:42Z


In [38]:
def cleanup (strava_df):
    """
    Clean and transform Strava activity DataFrame:
    - Convert start_date to datetime and extract date
    - Convert distance from meters to miles
    - Convert moving_time to minutes and add formatted time (hh:mm)
    - Convert elevation gain from meters to feet
    """
    # Change to datetime and create new date column
    strava_df.start_date = pd.to_datetime(strava_df.start_date) 
    strava_df['date'] = strava_df.start_date.dt.date
    # Change distance to miles
    strava_df['distance'] = strava_df['distance'] / 1609.34
    # Change moving time to minutes in new col: time
    strava_df['time'] = strava_df.moving_time/60
    # Convert to timedelta
    strava_df.moving_time = pd.to_timedelta(strava_df['moving_time'], unit='s')
    # Change to hours/minutes
    strava_df['moving_time'] = strava_df['moving_time'].apply(lambda td: f"{int(td.total_seconds() // 3600)}:{int((td.total_seconds() % 3600) // 60):02}")
    # Change to feet
    strava_df.total_elevation_gain = strava_df.total_elevation_gain*3.28084
    
    return strava_df

def convert_speed(row):
    if row['sport_type'] == 'Ride':
        return row['average_speed'] * 2.23694  # m/s to mph
    elif row['sport_type'] == 'Run':
        if row['average_speed'] > 0:
            pace_sec_per_mile = 1609.34 / row['average_speed']
            minutes = int(pace_sec_per_mile // 60)
            seconds = int(pace_sec_per_mile % 60)
            return f"{minutes}:{seconds:02d}"
        else:
            return "N/A"
    else:
        return None  # for other sports

In [39]:
# process data
cleanup (strava_df)

# add column: converted_speed -mph (ride) and -min/mile (run)
strava_df['converted_speed'] = strava_df.apply(convert_speed, axis=1)

# Reorder columns
new_order = ['id', 'name','date','sport_type',  'distance','moving_time', 'converted_speed', 
       'total_elevation_gain', 'average_heartrate', 'max_heartrate',
       'suffer_score', 'time','average_speed','start_date' ]

strava_df = strava_df[new_order]

# modify column names
strava_df.columns = ['id', 'name', 'date', 'sport', 'distance', 'time',
       'speed', 'elev_gain',
       'avg_HR', 'max_HR', 'suffer_score', 'time-minutes','average_speed',
       'start_date']

In [40]:
strava_df.head(2)

,id,name,date,sport,distance,time,speed,elev_gain,avg_HR,max_HR,suffer_score,time-minutes,average_speed,start_date
0,15048399185,Jamestown and eats at Lucky's Bakery,2025-07-08,Ride,37.5,2:37,14.3,2483.6,110.9,140.0,101.0,157.8,6.4,2025-07-08 14:44:02+00:00
1,15037283785,Old Stage,2025-07-07,Ride,27.7,1:58,14.0,1410.8,109.6,134.0,54.0,118.8,6.3,2025-07-07 12:38:38+00:00


### Gather activity data and build activity_df

In [41]:
# activity ID to inspect
activity_id = 15048399185

# endpoint URL
url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
# Set up Authorization header and make request
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    zones = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Build activity_df
activity_df = strava_df.loc[strava_df.id == activity_id, ['name','date']]
# add suffer score and total minutes to activity_df
activity_df['suffer_score'] = pd.DataFrame(zones[0]).iloc[0,0]

# create time_in_zones series
zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
time_in_zones = zone_series.apply(lambda z: z['time']/60)

# Create column names for all zones
column_names = [f'Zone{i+1}' for i in range(len(time_in_zones))]
# Convert to single-row DataFrame
zone_df = pd.DataFrame([time_in_zones.values], columns=column_names)

# set index of zond.df to match activity index before merge
zone_df.index = activity_df.index
activity_df = pd.merge(activity_df, zone_df,left_index=True, right_index=True)

# create moderate and intense summary columns
activity_df['moderate'] = activity_df.Zone1 + activity_df.Zone2
activity_df['intense']= activity_df.Zone3 + activity_df.Zone4 + activity_df.Zone5

In [42]:
activity_df

,name,date,suffer_score,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,intense
0,Jamestown and eats at Lucky's Bakery,2025-07-08,101.0,21.4,109.4,26.1,0.8,0.0,130.8,26.9


### Listing what is available 

In [ ]:
from pprint import pprint
n=0
for activity in activities:
    n+=1
    if n <= 1:
        pprint(activity)